In [1]:
import pydrake

本教程重点为: 多体动力学物理引擎(`MultibodyPlant`)和图形学渲染引擎(`SceneGraph`)

# 机器人描述文件

`drake` 支持 [URDF](http://wiki.ros.org/urdf) 和 [SSDFormat](http://sdformat.org/) 格式的文件, 他们的语法很相似. 虽然 `URDF` 是 `ROS` 中的标准格式, 但它缺少许多功能来描述更复杂的场景. 例如, `URDF` 只能指定运动学和动力学特性, 不支持闭链和摩擦等特性. 此外, 它不能指定非机器人的事物, 例如灯光、高度图等. `SDFormat` 是为了解决 URDF 的缺点而创建的. `SDFormat` 是从世界级到机器人级的所有内容的完整描述. 这种可扩展性使其更适合复杂的模拟. 鉴于此, 本教程使用 `SDFormat`. `link` 的 `mesh` 文件类型中, `OBJ(.OBJ)`目前是 `Drake` 支持最好的格式. `Meshlab` 可以将常见格式转换为`.obj`. 

In [25]:
import os
import numpy as np
from pydrake.geometry import Meshcat, MeshcatVisualizer, MeshcatVisualizerParams, Role
from pydrake.common import FindResourceOrThrow
from pydrake.visualization import ModelVisualizer
from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.systems.analysis import Simulator

In [3]:
# 启用图形学渲染引擎, 执行后会给一个 HTTP 链接, 打开链接会在浏览器中开一个 MeshCat 的标签
meshcat = Meshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# 查看模型文件

打开 `MeshCat` 的链接. `drake` 提供 `ModelVisualizer` 类交互式查看模型. 当我们制作自己的机器人描述文件从别的地方导出描述文件时, 这个类能为我们提供帮助. 下面将展示几个 drake 自带的模型

In [8]:
# 第一个例子是 KUKA iiwa 机械臂
# 注意 FindResourceOrThrow() 只能用于 drake 自带的模型文件, 不要用它打开自己制作的模型
iiwa7_model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")

# 创建一个模型可视化工具, 并把机械臂加进去
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(iiwa7_model_file)

# 测试模式(比如测试代码能不能跑通)下, 用户还来不及交互, 程序就执行结束了. 如果用户要和可视化工具交互, 就把 loop_once 参数设为 False
test_mode = True if "TEST_SRCDIR" in os.environ else False

# 启动交互式可视化工具, 在浏览器打开的 MeshCat 标签的 "Stop Running" 按钮停止交互
visualizer.Run(loop_once=test_mode)

Use Ctrl-C or click 'Stop Running' to quit


In [5]:
# 第二个例子是 Schunk WSG 夹爪
schunk_wsg50_model_file = FindResourceOrThrow("drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf")

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(schunk_wsg50_model_file)

visualizer.Run(loop_once=test_mode)

Use Ctrl-C or click 'Stop Running' to quit


# 制作自己的模型文件

In [7]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelFromFile("./cylinder.sdf", "sdf")

visualizer.Run(loop_once=test_mode)

Use Ctrl-C or click 'Stop Running' to quit


## visual 和 collision

回到 KUKA 机械臂的例子, 如果在 MeshCat 控制面板中勾选 drake/collision 复选框, 应该会看到围绕机械臂的白色框, 这些是 iiwa7_with_box_collision 中定义的碰撞几何体, 这些几何体通常用于运动规划或碰撞检查. 虽然可以把 mesh 同时用于视觉和碰撞, 但通过简单几何体近似复杂的mesh(如 KUKA 机械臂)可以极大地减少计算量. 检查两个圆柱体是否碰撞比检查两个不规则 mesh 更容易. 因此, 我们倾向于将 mesh 文件加载为视觉几何体, 但将各种简单几何体用作碰撞几何体. 

## 定义模型的碰撞几何体

由于碰撞几何体只是模型实际形状的近似, 因此我们希望近似合理接近实际. 经验法则是完全包裹实际形状, 但不要过度膨胀. 例如, 与试图用一个大的长方体覆盖 L 形模型相比, 使用两个长方体或圆柱体的效果会更好. 这是近似保真度和节省计算资源之间的平衡. 一般来说, 从实际形状的粗略近似开始, 看看是否引入了任何不希望的行为, 例如, 机器人认为它在碰撞, 但显然没有. 识别碰撞几何体中有问题的部分, 并用更精确的近似值替换它, 然后迭代. 

## 使用 mesh 作为碰撞几何体

在某些情况下需要为精确的碰撞几何体, 例如, 在对不规则形状的对象进行灵巧操作的情况下, 可以直接使用 mesh 作为碰撞几何体. 使用 OBJ 格式的 mesh 文件作为接触模型(即点接触模型)的碰撞几何体时, drake 会计算 mesh 的凸包并使用该凸包. 如果需要非凸碰撞几何体, 建议通过凸分解工具将 mesh 分解为各种凸形状. [V-HACD](https://github.com/kmammou/v-hacd/)上有许多类似的工具, [convex_comp_to_sdf](https://github.com/gizatt/convex_decomp_to_sdf)是 drake 使用的工具. 对于水弹性接触模型等更多信息, 请参阅[Hydroplastic](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)用户指南. \

此外 drake 提供了很多 URDF 和 SDFormat 之外的特性, 参考其[文档](https://drake.mit.edu/doxygen_cxx/group__multibody__parsing.html)

# 创建一个复杂的场景

我们将加载三个对象, 即饼干盒, 圆柱体和桌面. 饼干盒, 圆柱体一开始处于一定高度, 然后它们在重力作用下自由下落到桌面上. 

## 一些术语

主要说三个概念, `System`, `Diagram` 和 `Context`. `System` 就是所谓的系统, 这是一个更广义的概念, 它有输入输出接口, 且可以和其他系统相连, 比如 `MultibodyPlant` 和 `SceneGraph` 就属于系统. `Diagram` 是一个基元系统, 该系统由多个互连系统构成. `System` 和 `Diagram` 都有相应的 `Context` 来表示其状态, 并在仿真过程中更新. 如果给定了 `Diagram` 和相应的 `Context`, 仿真的结果就是确定的. 参考第一个教程 `01_dynamic_system.ipynb`

## 在场景中加载不同对象

在 `create_scene()` 函数中创建了一个 `pydrake.multibody.MultibodyPlant` 对象, 一个 `pydrake.multibody.SceneGraph` 对象, 一个 `pydrake.multibody.parsing.Parser` 对象. `Parser` 的用法之前说过, 不再赘述, 需要注意的是, 我们把桌子设置成相对于世界固定(fixed)或者说焊接(weld), 同时将饼干盒和圆柱体视为自由体(free body). `MultibodyPlant` 设置好后, 该函数将返回仿真器所需的 `Diagram`, 这个 `Diagram` 有默认的 `Context`. 

In [11]:
from pydrake.all import MultibodyPlant

In [36]:
def create_scene(sim_time_step=0.0001):
    # 清空 MeshCat
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    # scene_graph:
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    parser = Parser(plant)

    # 加载模型
    # 饼干盒
    cracker_box = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/003_cracker_box.sdf")
    parser.AddModels(cracker_box)
    # 桌面
    parser.AddModelFromFile('./table.sdf')
    parser.AddModelFromFile('./cylinder.sdf')

    # 将桌子和世界坐标系固定
    table_frame = plant.GetFrameByName('table_top_center')
    plant.WeldFrames(plant.world_frame(), table_frame)

    # 场景加载完毕
    plant.Finalize()

    # 用默认 context 计算桌子在世界坐标系下的位姿变换, 默认 context 并不是 diagram 用的 context
    plant_context = plant.CreateDefaultContext()

    # 设置自由物体的初始位姿
    cylinder = plant.GetBodyByName('cylinder_link')
    X_WorldTable = table_frame.CalcPoseInWorld(plant_context)
    X_TableCylinder = RigidTransform(RollPitchYaw(np.array([90, 0, 0]) * np.pi / 180), p=np.array([0, 0, 0.5]))
    X_WorldCylinder = X_WorldTable.multiply(X_TableCylinder)
    plant.SetDefaultFreeBodyPose(cylinder, X_WorldCylinder)

    cracker_box = plant.GetBodyByName("base_link_cracker")
    X_TableCracker = RigidTransform(RollPitchYaw(np.asarray([45, 30, 0]) * np.pi / 180), p=[0,0,0.8])
    X_WorldCracker = X_WorldTable.multiply(X_TableCracker)
    plant.SetDefaultFreeBodyPose(cracker_box, X_WorldCracker)

    # 添加可视化工具
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()

    return diagram, visualizer

In [37]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step):
    diagram, visualizer = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(5.0)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(sim_time_step=0.0001)


# 调试仿真和实物的差距

## 调试 inertial

仿真时间步长过小 (比如 < 0.001s), 或者收到关于 Delta>0 的报错或惯性矩阵无效的警告. 

## 调试 mass

固连到世界坐标系的对象无需指定质量. 但是, 一个质量为零的可移动对象会报错. 